In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [3]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

random_state = np.random.RandomState(0)
clf = OneVsRestClassifier(SVC(probability=True, random_state=random_state))

clf.fit(X, Y)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True,
  random_state=<mtrand.RandomState object at 0x7f87fa914288>,
  shrinking=True, tol=0.001, verbose=False),
          n_jobs=1)

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:2000])

WITH "SV_data" AS 
(SELECT "Values".sv_idx AS sv_idx, "Values".dual_coeff AS dual_coeff, "Values".sv_0 AS sv_0, "Values".sv_1 AS sv_1, "Values".sv_2 AS sv_2, "Values".sv_3 AS sv_3 
FROM (SELECT 0 AS sv_idx, -0.213687020099 AS dual_coeff, 7.0 AS sv_0, 3.2 AS sv_1, 4.7 AS sv_2, 1.4 AS sv_3 UNION ALL SELECT 1 AS sv_idx, -0.421524254498 AS dual_coeff, 4.9 AS sv_0, 2.4 AS sv_1, 3.3 AS sv_2, 1.0 AS sv_3 UNION ALL SELECT 2 AS sv_idx, -1.0 AS dual_coeff, 5.1 AS sv_0, 2.5 AS sv_1, 3.0 AS sv_2, 1.1 AS sv_3 UNION ALL SELECT 3 AS sv_idx, -0.202228133014 AS dual_coeff, 6.3 AS sv_0, 3.3 AS sv_1, 6.0 AS sv_2, 2.5 AS sv_3 UNION ALL SELECT 4 AS sv_idx, -0.0696490664691 AS dual_coeff, 5.8 AS sv_0, 2.8 AS sv_1, 5.1 AS sv_2, 2.4 AS sv_3 UNION ALL SELECT 5 AS sv_idx, -0.00453627448611 AS dual_coeff, 7.7 AS sv_0, 3.8 AS sv_1, 6.7 AS sv_2, 2.2 AS sv_3 UNION ALL SELECT 6 AS sv_idx, -0.37172099945 AS dual_coeff, 7.7 AS sv_0, 2.6 AS sv_1, 6.9 AS sv_2, 2.3 AS sv_3 UNION ALL SELECT 7 AS sv_idx, -0.289211353716 AS

## Execute the SQL Code

In [7]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [8]:
sql_output = pd.read_sql(lSQL , conn);


In [9]:
sql_output.sample(12)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
18,36,1.158289,-1.246089,-1.125822,None,None,None,None,None,None,0
24,4,1.246043,-1.323655,-1.130768,None,None,None,None,None,None,0
38,101,-1.081113,-0.999751,1.000028,None,None,None,None,None,None,2
25,30,1.112244,-1.136078,-1.174307,None,None,None,None,None,None,0
23,140,-1.083670,-1.850485,1.851072,None,None,None,None,None,None,2
107,32,1.173784,-1.280253,-1.050719,None,None,None,None,None,None,0
110,79,-1.089270,1.815510,-1.996133,None,None,None,None,None,None,1
132,88,-1.178317,1.559258,-1.571135,None,None,None,None,None,None,1
145,72,-1.108050,0.142555,-0.119465,None,None,None,None,None,None,1
74,112,-1.115643,-1.413520,1.458764,None,None,None,None,None,None,2


In [10]:
sql_output.Decision.value_counts()

2    52
0    50
1    48
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
#skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_log_proba = pd.DataFrame(columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
146,146,NaN,NaN,NaN,0.009755,0.057849,0.932396,NaN,NaN,NaN,2
60,60,NaN,NaN,NaN,0.008792,0.970796,0.020411,NaN,NaN,NaN,1
62,62,NaN,NaN,NaN,0.007691,0.990337,0.001972,NaN,NaN,NaN,1
104,104,NaN,NaN,NaN,0.009901,0.000574,0.989526,NaN,NaN,NaN,2
99,99,NaN,NaN,NaN,0.005929,0.990715,0.003356,NaN,NaN,NaN,1
12,12,NaN,NaN,NaN,0.974433,0.009637,0.015929,NaN,NaN,NaN,0
65,65,NaN,NaN,NaN,0.012198,0.984812,0.002989,NaN,NaN,NaN,1
53,53,NaN,NaN,NaN,0.004860,0.981251,0.013890,NaN,NaN,NaN,1
117,117,NaN,NaN,NaN,0.013967,0.007853,0.978180,NaN,NaN,NaN,2
80,80,NaN,NaN,NaN,0.005585,0.991169,0.003246,NaN,NaN,NaN,1


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.head(12)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
0,0,NaN,NaN,NaN,0.978103,0.007267,0.014629,NaN,NaN,NaN,...,-1.095265,-0.999559,0.999986,None,None,None,None,None,None,2
1,1,NaN,NaN,NaN,0.974982,0.010369,0.014648,NaN,NaN,NaN,...,-1.121845,-0.140479,0.091991,None,None,None,None,None,None,2
2,2,NaN,NaN,NaN,0.975701,0.007172,0.017127,NaN,NaN,NaN,...,-1.109687,-1.704922,1.730862,None,None,None,None,None,None,2
3,3,NaN,NaN,NaN,0.973226,0.010420,0.016354,NaN,NaN,NaN,...,1.098639,-1.174836,-1.064382,None,None,None,None,None,None,0
4,4,NaN,NaN,NaN,0.977579,0.006596,0.015824,NaN,NaN,NaN,...,-1.131664,-0.017093,0.072088,None,None,None,None,None,None,2
5,5,NaN,NaN,NaN,0.971892,0.015626,0.012481,NaN,NaN,NaN,...,-1.236336,1.562563,-1.557589,None,None,None,None,None,None,1
6,6,NaN,NaN,NaN,0.974794,0.007340,0.017866,NaN,NaN,NaN,...,1.171565,-1.276715,-1.088556,None,None,None,None,None,None,0
7,7,NaN,NaN,NaN,0.977791,0.008683,0.013526,NaN,NaN,NaN,...,-1.114607,0.944675,-0.933215,None,None,None,None,None,None,1
8,8,NaN,NaN,NaN,0.968398,0.011365,0.020236,NaN,NaN,NaN,...,1.190117,-1.265966,-1.106688,None,None,None,None,None,None,0
9,9,NaN,NaN,NaN,0.975229,0.010516,0.014255,NaN,NaN,NaN,...,-1.080438,-1.399291,1.445266,None,None,None,None,None,None,2


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
0,0,NaN,NaN,NaN,0.978103,0.007267,0.014629,NaN,NaN,NaN,...,-1.095265,-0.999559,0.999986,NaN,NaN,NaN,NaN,NaN,NaN,2
1,1,NaN,NaN,NaN,0.974982,0.010369,0.014648,NaN,NaN,NaN,...,-1.121845,-0.140479,0.091991,NaN,NaN,NaN,NaN,NaN,NaN,2
2,2,NaN,NaN,NaN,0.975701,0.007172,0.017127,NaN,NaN,NaN,...,-1.109687,-1.704922,1.730862,NaN,NaN,NaN,NaN,NaN,NaN,2
4,4,NaN,NaN,NaN,0.977579,0.006596,0.015824,NaN,NaN,NaN,...,-1.131664,-0.017093,0.072088,NaN,NaN,NaN,NaN,NaN,NaN,2
5,5,NaN,NaN,NaN,0.971892,0.015626,0.012481,NaN,NaN,NaN,...,-1.236336,1.562563,-1.557589,NaN,NaN,NaN,NaN,NaN,NaN,1
7,7,NaN,NaN,NaN,0.977791,0.008683,0.013526,NaN,NaN,NaN,...,-1.114607,0.944675,-0.933215,NaN,NaN,NaN,NaN,NaN,NaN,1
9,9,NaN,NaN,NaN,0.975229,0.010516,0.014255,NaN,NaN,NaN,...,-1.080438,-1.399291,1.445266,NaN,NaN,NaN,NaN,NaN,NaN,2
10,10,NaN,NaN,NaN,0.976073,0.009699,0.014228,NaN,NaN,NaN,...,-1.062877,-1.450070,1.465874,NaN,NaN,NaN,NaN,NaN,NaN,2
11,11,NaN,NaN,NaN,0.976041,0.009739,0.014220,NaN,NaN,NaN,...,-1.020644,-1.512184,1.499394,NaN,NaN,NaN,NaN,NaN,NaN,2
12,12,NaN,NaN,NaN,0.974433,0.009637,0.015929,NaN,NaN,NaN,...,-1.095011,-0.648889,0.713002,NaN,NaN,NaN,NaN,NaN,NaN,2
